In [ ]:
src_url = "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"
src_zip = "dakshina_dataset_v1.0.tar"
DATA_SRC="dakshina_dataset_v1.0/ta/lexicons"
DATA_TRAIN_SRC = "/ta.translit.sampled.train.tsv"
DATA_VAL_SRC = "/ta.translit.sampled.dev.tsv" 
DATA_TEST_SRC = "/ta.translit.sampled.test.tsv"
#TRAIN_IMAGES_PER_LABEL = 1000
#TEST_IMAGES_PER_LABEL = 200
BALANCED_SPLITS = {"train" : 900, "val" : 100}
PROJECT_NAME = "CS6910 ASSIGNMENT 3"
dataset='dakshina-dataset'

In [ ]:
%%capture
!curl -SL $src_url > $src_zip
!tar -xf $src_zip

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import pandas as pd
import keras
from keras.layers import Input, LSTM, Dense, Embedding, GRU, SimpleRNN, Dropout
from keras.models import Model

!pip3 install tensorflow -qqq
!pip3 install wandb -qqq
import wandb
!wandb login
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8MB 6.5MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 174kB 19.1MB/s 
     |████████████████████████████████| 133kB 22.0MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 

Uploading Data: only run once

In [ ]:

run = wandb.init(project=PROJECT_NAME, entity='cs6910krsrd',job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(dataset, type="raw_data")

raw_data_at.add_dir(DATA_SRC)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

wandb: Currently logged in as: krsrinivas (use `wandb login --relogin` to force relogin)


wandb: Adding directory to artifact (./dakshina_dataset_v1.0/ta/lexicons)... Done. 0.1s


Downloading Data

In [ ]:
run = wandb.init(project=PROJECT_NAME, entity='cs6910krsrd',job_type="download")

# Query W&B for an artifact and mark it as input to this run
artifact = run.use_artifact(dataset+':latest')

# Download the artifact's contents
artifact_dir = artifact.download()
run.finish()

In [ ]:
colnames=["ntv","rmn",'nAtt'] #native, romanized and number of attestations
df_train = pd.read_csv(artifact_dir + DATA_TRAIN_SRC,sep="\t",names=colnames,na_filter=False)
df_val = pd.read_csv(artifact_dir + DATA_VAL_SRC,sep="\t",names=colnames,na_filter=False)

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding and decoding space (LSTM/GRU/RNN)
num_samples = 10000  # Number of samples to train on.
embed_dim=16  #embedding size
n_encoder=2 #number of encoder layers 
n_decoder=2 #number of decoder layers
cell_type="LSTM"#"GRU" "RNN" # Cell type of encoder and decoder
do=0.2 #dropout

In [ ]:
input_texts = df_train.rmn.to_list()    # input words(romanized)
target_texts = df_train.ntv.apply(lambda s:'\t'+s+'\n').to_list() # target words(native)
input_characters = set(df_train.rmn.sum()) # input vocabulary (all english letters)
target_characters = set(df_train.ntv.sum()) # target vocabulary (all tamil letters)


input_characters = sorted(list(input_characters)) 
input_characters.append(' ')
target_characters = sorted(list(target_characters))
target_characters.append(' ')
target_characters.append('\t')
target_characters.append('\n')



num_encoder_tokens = len(input_characters)   # size of input vocabulary
num_decoder_tokens = len(target_characters)   # ize of target vocabulary
max_encoder_seq_length = max([len(txt) for txt in input_texts])  # max input word size
max_decoder_seq_length = max([len(txt) for txt in target_texts]) # max output word size

In [ ]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
 
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)]) #dict mapping input letters to integers
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)]) #dict mapping output letters to integers

encoder_input_data = np.zeros(                                              
    (len(input_texts), max_encoder_seq_length), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length,len(target_token_index)), dtype="float32"
)



In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[char]
    encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] =  target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1,target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :] = target_token_index[" "]
    decoder_target_data[i, t:,target_token_index[" "]] =  1.0


In [ ]:
CELL={"LSTM":LSTM,"GRU":GRU,"RNN":SimpleRNN}

INFO:tensorflow:Assets written to: s2s_tamil/assets


In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

model.save("s2s_tamil")

NameError: ignored

In [ ]:
def train():
  
    # Define an input sequence and process it.
    encoder_inputs=Input(shape=(None,))
    x = Embedding(num_encoder_tokens, embed_dim)(encoder_inputs)
    encoder_states = []
    for j in range(n_encoder)[:-1]:
      x, _, _ = CELL[cell_type](latent_dim,return_state=True,return_sequences=True)(x)
    x, state_h, state_c = CELL[cell_type](latent_dim,return_state=True,return_sequences=True)(x)
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(None,))
    x = Embedding(num_decoder_tokens, embed_dim)(decoder_inputs)
    for j in range(n_decoder)[:-1]:
      x,dh,dc = CELL[cell_type](latent_dim, return_sequences=True,return_state=True)(x, initial_state=encoder_states)

    dropout = Dropout(rate=do)
    x = dropout(x)
    decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(x)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(
        optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
    )
    model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
    )

    # Save model
    model.save("s2s_tamil")


In [ ]:
train()

In [ ]:
# Define an input sequence and process it.
encoder_inputs=Input(shape=(None,))
x = Embedding(num_encoder_tokens, embed_dim)(encoder_inputs)
encoder_states = []
for j in range(n_encoder)[:-1]:
  x, _, _ = CELL[cell_type](latent_dim,dropout=do,return_state=True,return_sequences=True)(x)
x, state_h, state_c = CELL[cell_type](latent_dim,dropout=do,return_state=True,return_sequences=True)(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
out_embed=Embedding(num_decoder_tokens, embed_dim)
embedded_word=out_embed(decoder_inputs)
x=embedded_word
output_layers = []
for j in range(n_decoder):
  output_layers.append(
      CELL[cell_type](latent_dim, return_sequences=True,return_state=True)
  )
  x,dh,dc = output_layers[-1](x, initial_state=encoder_states)

dropout = Dropout(rate=do)
x = dropout(x)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

# Save model
model.save("s2s_tamil")

Inference

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_tamil")

encoder_model = Model(encoder_inputs, encoder_states)

decoder_states_inputs = []
decoder_states = []
d_outputs=embedded_word
for j in range(n_decoder)[::-1]:
  current_state_inputs = [Input(shape=(latent_dim,)) for _ in range(2)]

  temp = output_layers[n_decoder-j-1](d_outputs, initial_state=current_state_inputs)

  d_outputs, cur_states = temp[0], temp[1:]
  decoder_states += cur_states
  decoder_states_inputs += current_state_inputs

decoder_outputs = decoder_dense(d_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = (encoder_model.predict(input_seq))
    states_value=states_value*n_decoder

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index["\t"]

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        #print("a",np.shape([target_seq] + states_value))
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [ ]:
print(decode_sequence(encoder_input_data[10:11]))
print(input_texts[10])

பிலிமஸ்

filims


In [ ]:
def calcValAcc():
  

In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)